In [40]:
import numpy as np
import random
import string
import math
from collections import defaultdict
import json
import os

In [2]:
def gen_word(word_length):
    word_len = np.random.randint(*word_length)
    return ''.join(random.sample(string.ascii_lowercase, word_len))

In [3]:
topic_size = 6
num_topics = 20

topics = [
    [gen_word((2,6)) for _ in range(topic_size)] for _ in range(num_topics)
]

In [4]:
topics

[['srpq', 'mfvt', 'oicel', 'so', 'ov', 'rmg'],
 ['hs', 'cjhrt', 'gr', 'nbtz', 'vnbf', 'pjn'],
 ['migw', 'gx', 'tyhfl', 'vfim', 'qtd', 'es'],
 ['gfsrc', 'dvr', 'xyn', 'nzr', 'ks', 'wnk'],
 ['khvd', 'sd', 'xh', 'xurpb', 'tbzh', 'xpfh'],
 ['akqw', 'ybov', 'bacj', 'vawin', 'lirjt', 'yigk'],
 ['kmbd', 'lvhm', 'wzr', 'em', 'typm', 'cxj'],
 ['abzxw', 'qeg', 'zsl', 'ra', 'aowf', 'rpb'],
 ['arg', 'em', 'kic', 'by', 'gin', 'ovq'],
 ['fuwyi', 'av', 'hmb', 'eurjf', 'bur', 'ad'],
 ['kwejo', 'ku', 'pgns', 'ztu', 'snm', 'edpmj'],
 ['uoyxf', 'hgr', 'xg', 'swb', 'emns', 'hd'],
 ['bvl', 'eh', 'kb', 'bf', 'rn', 'rnlg'],
 ['dwbn', 'atds', 'dklyp', 'pmj', 'arsd', 'tcvma'],
 ['ufp', 'omh', 'ex', 'twygo', 'rh', 'vabxq'],
 ['cenzb', 'ak', 'lps', 'hkgm', 'npkji', 'ozv'],
 ['ozhkl', 'ib', 'zxtrg', 'qkpg', 'yw', 'rp'],
 ['nr', 'fdbv', 'djt', 'jcfvh', 'lumt', 'bs'],
 ['qs', 'ukhbn', 'zxy', 'abgc', 'omlt', 'yajgk'],
 ['mv', 'kytnc', 'bjfs', 'cfs', 'dqmec', 'opkbh']]

In [5]:
def generate_sentence(m_topics):
    sent = []
    prefix_topics = 1
    suffix_topics = 1
    
    for i in range(prefix_topics):
        sent += random.sample(topics[random.choice(range(num_topics))], random.randint(0,2))
    
    mention = random.choice(random.choice(topics))
    
    sent += random.sample(topics[random.choice(m_topics)], random.randint(0,2)) + ["\t", mention, "\t"] + random.sample(topics[random.choice(m_topics)], random.randint(0,2))
    
    for i in range(suffix_topics):
        sent += random.sample(topics[random.choice(range(num_topics))], random.randint(0,2))
      
    return sent
    

In [6]:
m_topics = random.choices(range(num_topics), k=2)
m_topics, generate_sentence(m_topics)

([4, 12], ['khvd', '\t', 'oicel', '\t', 'swb'])

In [7]:
def generate_to_file(file_path, count=1000):
    with open(file_path, 'w') as out:
        for i in range(count):
            m_topics = random.choices(range(num_topics), k=2)
            topics_str = "_".join(map(str, m_topics))
            for j in range(random.randint(2, 4)):
                sent = generate_sentence(m_topics)
                out.write(topics_str + "\t" + ' '.join(sent) + '\n')


In [26]:
generate_to_file('../data/fake_data_train.tsv', count=10000)
generate_to_file('../data/fake_data_valid.tsv', count=500)          
generate_to_file('../data/fake_data_test.tsv', count=500) 

In [9]:
with open('../data/fake_ft_data.tsv', 'w') as out:
    for i in range(1000):
        m_topics = random.choices(range(num_topics), k=2)
        topics_str = "_".join(map(str, m_topics))
        for j in range(random.randint(2, 4)):
            sent = generate_sentence(m_topics)
            out.write(' '.join(sent) + '\n')

In [10]:
#!pip install git+https://github.com/facebookresearch/fastText.git

In [11]:
import fastText

In [12]:
model = fastText.train_unsupervised(
    input='../data/fake_ft_data.tsv', minCount=0, bucket=1000, dim=30
)

In [13]:
len(model.get_words())

120

In [14]:
model.save_model("../data/fake_ft_model.bin")

In [15]:
from gensim.models import FastText
from gensim.models.utils_any2vec import ft_ngram_hashes
from gensim.models.utils_any2vec import compute_ngrams

In [16]:
ft = FastText.load_fasttext_format("../data/fake_ft_model.bin")

/home/generall/sources/Sci/ml/allen_evn/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [17]:
ft.save('../data/gensim_fake_ft.model')

In [18]:
ft2 = FastText.load('../data/gensim_fake_ft.model')

In [48]:
ft2.min_n

/home/generall/sources/Sci/ml/allen_evn/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `min_n` (Attribute will be removed in 4.0.0, use wv.min_n instead).
  """Entry point for launching an IPython kernel.


3

In [24]:
topics_mapping = {}
for idx, _ in enumerate(topics):
    for idx2, _ in enumerate(topics):
        topics_mapping[f"{idx}_{idx2}"] = [str(idx), str(idx2)]

In [25]:
with open('../data/category_mapping.json', 'w') as out:
    json.dump(topics_mapping, out)

In [31]:
!split -n l/10 --verbose ../data/fake_data_train.tsv ../data/fake_data_train.tsv_

creating file '../data/fake_data_train.tsv_aa'
creating file '../data/fake_data_train.tsv_ab'
creating file '../data/fake_data_train.tsv_ac'
creating file '../data/fake_data_train.tsv_ad'
creating file '../data/fake_data_train.tsv_ae'
creating file '../data/fake_data_train.tsv_af'
creating file '../data/fake_data_train.tsv_ag'
creating file '../data/fake_data_train.tsv_ah'
creating file '../data/fake_data_train.tsv_ai'
creating file '../data/fake_data_train.tsv_aj'


In [36]:
!pip install google-cloud-storage

    100% |████████████████████████████████| 71kB 653kB/s 
  Using cached https://files.pythonhosted.org/packages/0c/f2/3c225e7a69cb27d283b68bff867722bd066bc1858611180197f711815ea5/google_cloud_core-0.29.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 71kB 10.3MB/s 
  Using cached https://files.pythonhosted.org/packages/e2/5d/4bc5c28c252a62efe69ed1a1561da92bd5af8eca0cdcdf8e60354fae9b29/google_resumable_media-0.3.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c5/9b/ed0516cc1f7609fb0217e3057ff4f0f9f3e3ce79a369c6af4a6c5ca25664/google_auth-1.6.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/39/2b/d87fc2369242bd743883232c463f28205902b8579cb68dcf5b11eee1652f/cachetools-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/da/98/8ddd9fa4d84065926832bcf2255a2b69f1d03330aa4d1c49cc7317ac888e/pyasn1_modules-0.2.4-py2.py3-none-any.whl
  Stored in directory: /home/generall/.cache/pip/wheels/b8